And here I start

In [215]:
import pandas as pd
import requests
import numpy as np
import io

Here, I create links for the data to download. There is no data for 2011, so we have to omit that one. We convert everything into pd.DataFrame, so that we have all pandas functionality right-away.

In [216]:
download_link = 'https://rsf.org/sites/default/files/import_classement/2000.csv'
links = []
for i in range(2, 26):
    if i == 11:
        continue
    links.append(download_link.replace('2000', f'{2000 + i}'))
links = pd.Series(links)
links[0]

'https://rsf.org/sites/default/files/import_classement/2002.csv'

Load files with requests and reading them. I had to create file-objects, cos read_csv accepts only those, it can't read strings directly. RSF decided to use semi-columns as separators

In [217]:
files = links.apply(lambda x: requests.get(x))
files = files.apply(lambda x : pd.read_csv(io.StringIO(x.text), sep=';'))
print(files[0])

     ï»¿Year (N)  ISO  Rank N Score N  Score N without the exactions  \
0           2002  FIN       1     0,5                            NaN   
1           2002  ISL       2     0,5                            NaN   
2           2002  NOR       3     0,5                            NaN   
3           2002  NLD       4     0,5                            NaN   
4           2002  CAN       5    0,75                            NaN   
..           ...  ...     ...     ...                            ...   
134         2002  BTN     135   90,75                            NaN   
135         2002  TKM     136    91,5                            NaN   
136         2002  MMR     137   96,83                            NaN   
137         2002  CHN     138      97                            NaN   
138         2002  PRK     139    97,5                            NaN   

     Score N with the exactions  Score exactions  Rank N-1  Score N-1  \
0                           NaN              NaN       NaN    

Now, joining 

In [276]:
data = pd.concat(list(files))
print(data.columns)

Index(['ï»¿Year (N)', 'ISO', 'Rank N', 'Score N',
       'Score N without the exactions', 'Score N with the exactions',
       'Score exactions', 'Rank N-1', 'Score N-1', 'Rank evolution',
       'FR_country', 'EN_country', 'ES_country', 'AR_country', 'FA_country',
       'Zone', 'ï»¿ISO', 'Score', 'Rank', 'Political Context', 'Rank_Pol',
       'Economic Context', 'Rank_Eco', 'Legal Context', 'Rank_Leg',
       'Social Context', 'Rank_Soc', 'Safety', 'Rank_Saf', 'Country_EN',
       'Country_FR', 'Country_ES', 'Country_AR', 'Country_FA', 'Year (N)',
       'Country_PT', 'Score evolution', 'Situation', 'Score 2025'],
      dtype='object')


Dropping obviously useless columns 

In [277]:
data = data.rename(columns={'ï»¿Year (N)': 'Year'})
data = data.drop(['Country_FR', 'Country_ES', 'Country_AR', 'Country_FA', 'Country_PT', 
           'FR_country', 'ES_country', 'AR_country', 'FA_country', 'Zone', 'ISO', 'ï»¿ISO',
           'Rank_Eco', 'Rank_Pol','Rank_Leg', 'Rank_Soc', 
           'Rank_Saf', 'Score N', 'Score evolution', 'Score 2025', 'Situation'], axis=1)
print(data.columns)

Index(['Year', 'Rank N', 'Score N without the exactions',
       'Score N with the exactions', 'Score exactions', 'Rank N-1',
       'Score N-1', 'Rank evolution', 'EN_country', 'Score', 'Rank',
       'Political Context', 'Economic Context', 'Legal Context',
       'Social Context', 'Safety', 'Country_EN', 'Year (N)'],
      dtype='object')


Standardize columns, convert the df to more efficient data types. Unite year and rank columns that were named differently into one. Dropping some more obsolete columns, such as used 'year_(n)' duplicate, rank and score of prev year, evolution, score and others.

Main job of the platform will be to compare rankings in a nice and visual way, using more detailed info as needed (as economic context and other "contexts"). Those detailed parameters will me measured in scores, as it's more relevant to compare the economic context of a country at some absolute scale, comparing with itself, and not with how it changes in others.

In [278]:
data = data.convert_dtypes()
data.columns = data.columns.str.lower().str.replace(' ', '_')

data['year'] = data['year'].where(data['year'].notna(), data['year_(n)'])
data['rank_n'] = data['rank_n'].where(data['rank_n'].notna(), data['rank'])
data['en_country'] = data['en_country'].where(data['en_country'].notna(), data['country_en'])

data = data.drop(['year_(n)', 'rank', 'rank_n-1', 'score_n-1', 'rank_evolution', 'score', 
                   'score_exactions', 'score_n_without_the_exactions', 
                   'score_n_with_the_exactions', 'country_en'], axis=1)
data = data.rename(columns={'rank_n': 'rank'})


Last touch of standartization in columns naming

In [284]:
new_col_names = []
for name in data.columns:
    if name == 'safety':
        new_col_names.append('safety_factor')
        continue
    new_col_names.append(name.replace('context', 'factor'))
data.columns = new_col_names

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4020 entries, 0 to 179
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              4020 non-null   object
 1   rank              4020 non-null   Int64 
 2   en_country        4020 non-null   string
 3   political_factor  720 non-null    string
 4   economic_factor   720 non-null    string
 5   legal_factor      720 non-null    string
 6   social_factor     720 non-null    string
 7   safety_factor     720 non-null    string
dtypes: Int64(1), object(1), string(6)
memory usage: 286.6+ KB
None
